In [23]:
import os
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv("./Data/preparation.csv")

In [25]:
df = df[["image","product_category_tree_target"]]

In [26]:
df["image"] = df["image"].apply(lambda x:"./Data/Images/"+x)

In [27]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# Define the ratio for the split
split_ratio = 0.8  # 80% training, 20% testing

# Calculate the number of rows for training and testing
total_rows = df.shape[0]
train_rows = int(split_ratio * total_rows)
test_rows = total_rows - train_rows

# Split the DataFrame into training and testing sets
train_data = df.iloc[:train_rows, :]["image"].to_frame()
train_data_target = df.iloc[:train_rows, :]["product_category_tree_target"].to_frame()
test_data = df.iloc[train_rows:, :]["image"].to_frame()
test_data_target = df.iloc[train_rows:, :]["product_category_tree_target"].to_frame()


label = preprocessing.LabelEncoder()
y = label.fit_transform(train_data_target['product_category_tree_target'])
y = to_categorical(y)
print(y[:5])

[[0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]


In [28]:
labels = label.classes_
print(labels)
len_cat = len(labels)

['Baby Care' 'Beauty and Personal Care' 'Computers'
 'Home Decor & Festive Needs' 'Home Furnishing' 'Kitchen & Dining'
 'Watches']


In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
# Créez un générateur d'images pour prétraiter les images
image_size = (224, 224)  # Ajustez la taille d'image à celle attendue par EfficientNet
batch_size = 32
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Mise à l'échelle des pixels entre 0 et 1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Utilisé pour diviser les données en ensemble d'entraînement et de validation
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='image',
    y_col='product_category_tree_target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='image',
    y_col='product_category_tree_target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Entraînez le modèle
epochs = 10  # Nombre d'époques pour l'entraînement (ajustez selon vos besoins)
# Créez un modèle EfficientNetB5 pré-entraîné
effnet_model = EfficientNetB5(
    include_top=False,  # Exclut la couche de classification finale
    weights='imagenet',  # Utilise les poids pré-entraînés sur ImageNet
    input_shape=(224, 224, 3),  # Remplacez par la forme de vos images d'entrée
    pooling='avg'  # Vous pouvez changer cela en 'max' ou laisser tel quel
)

# Ajoutez une nouvelle couche de classification personnalisée
 # Remplacez par le nombre de classes de votre problème
x = Dense(len_cat, activation='softmax')(effnet_model.output)

# Créez le modèle complet
model = Model(inputs=effnet_model.input, outputs=x)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Afficher un résumé du modèle
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs
)

# Évaluez le modèle sur l'ensemble de test
test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    x_col='image',
    y_col='product_category_tree_target',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)

Found 672 validated image filenames belonging to 7 classes.
Found 168 validated image filenames belonging to 7 classes.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_2[0][0]'